In [1]:
import sys
sys.path.append('../')

import torch
from models.yolo import ZoominHead,MultiHead,Detect, Model
from models.experimental import attempt_load
import copy
from models.tf import TFModel
import keras
import tensorflow as tf

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-05 14:30:59.069736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 14:30:59.305286: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 14:30:59.878005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc

In [2]:
yolo_model = attempt_load('../runs/train/prune-66-finetune-carpart-add-car-v2-anchor-tune-3-od-od/weights/best.pt')
zoomin_checkpoint = torch.load('zoomin_model.pth',map_location='cpu')
zoomin_model = ZoominHead()
zoomin_model.load_state_dict(zoomin_checkpoint)

Fusing layers... 
YOLOv5s-anchortune-3 summary: 157 layers, 6040633 parameters, 0 gradients, 13.1 GFLOPs


<All keys matched successfully>

In [ ]:
ori_model = copy.deepcopy(yolo_model)

In [6]:
a = yolo_model.model[24]
print(a)
# print(a.i,a.f,a.type,a.np)
yolo_model.model[24]([torch.randn(1,85,320,320),torch.randn(1,154,320,320),torch.randn(1,229,320,320)])

Detect(
  (m): ModuleList(
    (0): Conv2d(85, 33, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(154, 33, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2d(229, 33, kernel_size=(1, 1), stride=(1, 1))
  )
)


(tensor([[[ 3.93070e+00,  3.48126e+00,  2.08177e+01,  ...,  1.41654e-02,  1.45517e-02,  1.94451e-02],
          [ 1.36284e+01,  3.87928e+00,  1.58100e+01,  ...,  1.89717e-02,  1.78646e-02,  8.68124e-03],
          [ 1.85373e+01, -1.79207e+00,  1.74234e+01,  ...,  4.69012e-02,  2.54765e-02,  2.27858e-02],
          ...,
          [ 1.01649e+04,  1.02268e+04,  1.28783e+02,  ...,  1.79133e-02,  9.06027e-03,  4.47783e-02],
          [ 1.01843e+04,  1.02230e+04,  1.29324e+02,  ...,  1.36238e-02,  1.08309e-02,  6.88399e-02],
          [ 1.02406e+04,  1.02215e+04,  1.86100e+02,  ...,  1.36667e-02,  1.08170e-02,  5.72843e-02]]]),
 [tensor([[[[[-0.01732, -0.12987,  0.56635,  ..., -4.24269, -4.21539, -3.92052],
             [ 0.41286, -0.03018,  0.22464,  ..., -3.94565, -4.00691, -4.73787],
             [-0.36984, -1.83204,  0.33758,  ..., -3.01167, -3.64419, -3.75857],
             ...,
             [-0.29977, -0.59321,  0.32115,  ..., -3.55463, -2.80064, -4.23598],
             [-0.72787, -0.7

In [7]:
sample1 = torch.randn(1,85,320,320)
sample2 = torch.randn(1,154,320,320)
sample3 = torch.randn(1,229,10,10)

In [8]:
# [i.sum() for i in sample]

In [9]:
test = yolo_model.model[24]([sample1,sample2,sample3])
test[0].sum()

tensor(8.01566e+08)

In [10]:
class MultiHead(Detect):
    def __init__(self, detect_module,zoomin_module):
        super().__init__(detect_module.nc, detect_module.anchors, detect_module.ch, detect_module.inplace)
        self.detect = Detect.forward
        self.m = detect_module.m
        self.nl = detect_module.nl
        self.na = detect_module.na
        self.grid = detect_module.grid
        self.anchor_grid = detect_module.anchor_grid
        self.stride = detect_module.stride
        
        self.zoomin_head = zoomin_module
        self.sofmax = torch.nn.Softmax(dim=1)
        
        self.f = detect_module.f
        self.i = detect_module.i
        self.type  = str(type(self))[8:-2].replace('__main__.','')
        self.np = detect_module.np + sum([n.numel() for n in zoomin_module.parameters()])
        

    def forward(self, x):
        p = self.zoomin_head(x[-1])
        p = self.sofmax(p)[:,0] 
        
        x = self.detect(self,x)
        return x[0], p
    
    def load_from(self,detect_module):
        self.m = detect_module.m

In [8]:
detect_m = yolo_model.model[24]
detect_m.ch = [85,154,229]
mh = MultiHead(detect_m,zoomin_model)
mh.eval()
yolo_model.model[24] = mh

In [4]:
print([n.numel() for n in zoomin_model.parameters()])
print([n.numel() for n in detect_m.parameters()])

[229, 1, 625, 25, 50, 2]
[2805, 33, 5082, 33, 7557, 33]


In [5]:
sample = torch.randn(1,3,320,320)

In [9]:
out,z = yolo_model(sample)
print(out.sum(),z)

tensor(829226.43750) tensor([1.88164e-07], grad_fn=<SelectBackward>)


In [86]:
ori_model(sample)[0].sum()
print(ori_model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 25, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(25, 55, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(55, 24, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(55, 27, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(51, 40, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(24, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [10]:
ckpt = {'model':yolo_model}
torch.save(ckpt,'yolo_zoomin_head_torch.pt')

In [19]:
detect_m = yolo_model.model[24]
print(detect_m.nc)
print(detect_m.no)
print(detect_m.nl)
print(detect_m.na)
print([i.sum() for i in detect_m.grid])
print([i.sum() for i in detect_m.anchor_grid])
print(detect_m.anchors)
print(detect_m.stride)

28
33
3
1
[tensor(32563200.), tensor(32563200.), tensor(32563200.)]
[tensor(2454400.), tensor(8654157.), tensor(26728156.)]
tensor([[[1.59863, 1.39746]],

        [[2.53125, 2.75195]],

        [[4.67969, 3.47852]]])
tensor([ 8., 16., 32.])


In [20]:
print(mh.nc)
print(mh.no)
print(mh.nl)
print(mh.na)
print([i.sum() for i in mh.grid])
print([i.sum() for i in mh.anchor_grid])
print(mh.anchors)
print(mh.stride)

28
33
3
1
[tensor(32563200.), tensor(32563200.), tensor(32563200.)]
[tensor(2454400.), tensor(8654157.), tensor(26728156.)]
tensor([[[1.59863, 1.39746]],

        [[2.53125, 2.75195]],

        [[4.67969, 3.47852]]])
None


In [3]:
yolo_model.yaml

{'nc': 28,
 'depth_multiple': 0.33,
 'width_multiple': 0.5,
 'anchors': [[16, 30], [62, 45], [156, 198]],
 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 3, 'C3', [128]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 6, 'C3', [256]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 9, 'C3', [512]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 3, 'C3', [1024]],
  [-1, 1, 'SPPF', [1024, 5]]],
 'head': [[-1, 1, 'Conv', [512, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [256, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 3, 'C3', [256, False]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 14], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 10], 1, 'Concat', [1]],
  [-1, 3, 'C3', [1024, False]],
  [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]],
 'ch': 3}

In [10]:
tfmodel = TFModel(cfg=yolo_model.yaml,model=yolo_model,nc=yolo_model.nc,imgsz=(320,320))


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

conv :  ListWrapper([25, 3, 6, 6])
conv :  ListWrapper([55, 25, 3, 3])
conv :  ListWrapper([24, 55, 1, 1])
conv :  ListWrapper([27, 55, 1, 1])
conv :  ListWrapper([40, 51, 1, 1])
conv :  ListWrapper([32, 24, 1, 1])
conv :  ListWrapper([24, 32, 3, 3])
conv :  ListWrapper([117, 40, 3, 3])
conv :  ListWrapper([58, 117, 1, 1])
conv :  ListWrapper([60, 117, 1, 1])
conv :  ListWrapper([110, 118, 1, 1])
conv :  ListWrapper([64, 58, 1, 1])
conv :  ListWrapper([58, 64, 3, 3])
conv :  ListWrapper([64, 58, 1, 1])
conv :  ListWrapper([58, 64, 3, 3])
conv :  ListWrapper([255, 110, 3, 3])
conv :  ListWrapper([127, 255, 1, 1])
conv :  ListWrapper([127, 255, 1, 1])
conv :  ListWrapper([252, 254, 1, 1])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([128, 127, 1, 1])
conv :  ListWrapper([127, 128, 3, 3])
conv :  ListWrapper([512, 252, 3, 3])
conv :  ListWrapper([254, 512, 1, 1])


 13                -1  1    361984  models.common.C3                        [512, 256, 1, False]          
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]              
 15                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 16           [-1, 4]  1         0  models.common.Concat                    [1]                           
 17                -1  1     90880  models.common.C3                        [256, 128, 1, False]          
 18                -1  1    147712  models.common.Conv                      [128, 128, 3, 2]              
 19          [-1, 14]  1         0  models.common.Concat                    [1]                           
 20                -1  1    296448  models.common.C3                        [256, 256, 1, False]          
 21                -1  1    590336  models.common.Conv                      [256, 256, 3, 2]              
 22          [-1, 10]  1         0  m

conv :  ListWrapper([128, 128, 1, 1])
conv :  ListWrapper([128, 128, 3, 3])
conv :  ListWrapper([120, 232, 1, 1])
debug concat :  256
conv :  ListWrapper([60, 230, 1, 1])
conv :  ListWrapper([62, 230, 1, 1])
conv :  ListWrapper([85, 122, 1, 1])
conv :  ListWrapper([64, 60, 1, 1])
conv :  ListWrapper([60, 64, 3, 3])
conv :  ListWrapper([120, 85, 3, 3])
debug concat :  256
conv :  ListWrapper([107, 240, 1, 1])
conv :  ListWrapper([113, 240, 1, 1])
conv :  ListWrapper([154, 220, 1, 1])
conv :  ListWrapper([128, 107, 1, 1])
conv :  ListWrapper([107, 128, 3, 3])
conv :  ListWrapper([207, 154, 3, 3])
debug concat :  512
conv :  ListWrapper([217, 458, 1, 1])
conv :  ListWrapper([224, 458, 1, 1])
conv :  ListWrapper([229, 441, 1, 1])
conv :  ListWrapper([256, 217, 1, 1])
conv :  ListWrapper([217, 256, 3, 3])
weight :  ModuleList(
  (0): Conv2d(85, 33, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(154, 33, kernel_size=(1, 1), stride=(1, 1))
  (2): Conv2d(229, 33, kernel_size=(1, 1), stride=(

In [11]:
tfmodel.model.layers[24]

In [13]:
im = tf.zeros((1,320,320 , 3))
tfmodel.predict(im)

AttributeError: 'NoneType' object has no attribute 'shape'

<tf.Tensor: shape=(1, 320, 320, 3), dtype=float32, numpy=
array([[[[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0]],

        [[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0]],

        [[          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           0],
         ...,
         [          0,           0,           0],
         [          0,           0,           0],
         [          0,           0,           